In [5]:
import os
import sys
sys.path.append('../va_pipeline/')
import json
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

from calculate_accuracy import *
# from drawbox import *
from process import *
from framedif import *
from sensor import exec_file



In [31]:
test_dir = '../samples/testing/videos/'

# for vid in list(os.listdir(test_dir)):
#     print(os.path.join(test_dir, vid))
    # cap = cv2.VideoCapture(os.path.join(test_dir, vid))
queue = [cap.read()[1], cap.read()[1], cap.read()[1]]

ret, prev = cap.read()
frame_no = 1
frames_processed = 1

diffs = []
while frame_no <= 250:
    ret, frame = cap.read()

    if not ret:
        break
    
    queue.pop(0)
    queue.append(frame)
    frame = get_diff(queue[0], queue[1], queue[2])
    
    frame_no += 1

# df[vid] = diffs
cap.release()
df.plot()


../samples/testing/videos/large_fast.mp4
../samples/testing/videos/large_slow.mp4
../samples/testing/videos/small_fast.mp4
../samples/testing/videos/small_slow.mp4


In [6]:
file = './test_results/config_testing/resolution/stats/largefast_1280_720_25_stats.txt'
df = pd.read_csv(file, \
                    sep=': ', header=None, engine='python').set_index(0)
df = df.transpose().reset_index(drop=True)

config_params = file.split('_')

# Setting columns based on params
df['Video'], df['Width (px)'], df['Height (px)'], df['Frame Rate (fps)'] = \
config_params[0], config_params[1], config_params[2], config_params[3]

# Add columns
df['Resolution (px)'] = int(df['Width (px)']) * int(df['Height (px)'])
df['Resolution (1000s of px)'] = df['Resolution (px)'] / 1000
df['Energy (1000s of J)'] = df['energy'] / 1000
df['Energy (J) per frame'] = df['energy'] / 250


# Count # of detections
inf_path = str(os.path.splitext(file)[0])[:-5] + "inference.csv"
with open(inf_path, 'r') as f:
    lines = f.readlines()
    df['Detections'] = len(lines) - 1  # subtract the first line, which is cols of df
    


C:\Users\adamf\AppData\Local\Temp\ipykernel_8556\68626124.py:13: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  df['Resolution (px)'] = int(df['Width (px)']) * int(df['Height (px)'])


ValueError: invalid literal for int() with base 10: 'results/config'